In [1]:
import locussdk as locus
import pandas as pd 
import numpy as np
import folium
import re
import json
import folium

pnames = {"K":"Ontario",
"L":"Ontario",
"M":"Ontario",
"N":"Ontario",
"P":"Ontario",
"A":"Newfoundland and Labrador",
"B":"Nova Scotia",
"C":"Prince Edward Island",
"E":"New Brunswick",
"G":"Quebec",
"H":"Quebec",
"J":"Quebec",
"R":"Manitoba",
"S":"Saskatchewan",
"T":"Alberta",
"V":"British Columbia",
"X":"Nunavut",
"Y":"Yukon"}

provinces = '../_other/canada_provinces.geojson'

gzp = 193 #percent
locus.execute_saved_query(gzp)

In [33]:
#data preparation
gzp = locus.get_saved_query(gzp)
gzp['geo_id'] = gzp['geo_ca_postalcode'].str[:3]
fsa = gzp.groupby('geo_id', as_index = False).agg({'percent': 'mean'})
fsa['region'] = fsa['geo_id'].str[:1].map(pnames)
toronto = fsa[fsa['geo_id'].str[:1].map({'M':'Toronto'}).isin(['Toronto'])]
d = {'name': ["Old Toronto (Central)", "East End", 
              "North End", "West End", "East York", 
              "Etobicoke", "North York", "Scarborough"],
    'LAT': [43.655497895189264, 43.6807350107613,43.69752915179648, 
            43.66421565604445, 43.703072429165026, 43.63642392282264,
            43.767526456678716,43.77369091510212], 
    'LON': [-79.38174276808252,-79.33297321306567,-79.40167055351647,
            -79.46993208600004, -79.34022669598363, -79.54409721502077,
            -79.4289540616619, -79.24040959957513]} 
areas = pd.DataFrame(data = d)

#Toronto
geojson = locus.get_geojson(list(toronto.copy()['geo_id']), 'fsa')
for item in geojson['features']:
    try: 
        item['properties']['name'] = re.search(r'_(.*?)_' ,item['properties']['name']).group(1)
    except:
        continue

    
fsa_map = folium.Map(location=[43.6487773,-79.3878521], zoom_start=11, tiles = 'cartodbpositron')
tiles = ['stamenwatercolor','cartodbpositron','openstreetmap','stamenterrain']

for tile in tiles:
    folium.TileLayer(tile).add_to(fsa_map)

choropleth = folium.Choropleth(
    geo_data=geojson,
    name='Average % of Gen Z',
    data= toronto.copy(),
    columns=['geo_id', 'percent'],
    key_on='feature.properties.name',
    fill_color ='Reds',
    fill_opacity = 0.75,
    line_opacity = 0.2,
    legend_name='Percent, %',
).add_to(fsa_map)

style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False))

for i in range(0,len(areas)):
    folium.Marker(
        location=[areas.iloc[i]['LAT'], areas.iloc[i]['LON']],
        popup=areas.iloc[i]['name'],
        icon=folium.DivIcon(html=f"""<div style="font-family: roboto; color: #444444; font-size:16px; text-shadow: 1px 1px #ffffff"><b>{areas.iloc[i]['name']}</b></div>""")
    ).add_to(fsa_map)

#FloatImage(, top=40, left=65).add_to(m)

title_html = '''
             <h1 align="center" style="font-size:24px"><p><b>Distribution of Generation Z representatives in the city of Toronto, Ontario</b><i></i></p></h1>
             ''' #(Downtown & Etobicoke)
fsa_map.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl().add_to(fsa_map)


fsa_map

### _End of Notebook_